# Flights Price Analysis 

The goal of this notebook is to run some analysis on a dataset that contains one-way flights found on Expedia between 2022-04-16 and 2022-10-05 (you can find it at this [link](https://www.kaggle.com/datasets/dilwong/flightprices)).

In [1]:
//val bucketname = "unibo-bd2223-paolopenazzi"
val bucketname = "unibo-bd2223-vfolin"

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1677139979838_0001,spark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

bucketname: String = unibo-bd2223-vfolin


## Data Preparation

The following columns will be kept and created:
- flightID: Identifier for the flight.
- searchDate: Date the record was obtained from Expedia.
- searchMonth: Month the record was obtained from Expedia.
- searchDay: Day the record was obtained from Expedia.
- flightDate: The date of the flight.
- flightMonth: The month of the flight.
- flightDay: The day of the flight.
- startingAirport: 3-letter code for the starting airport.
- destinationAirport: 3-letter code for the destination airport.
- duration: Travel duration in minutes.
- isEconomy: Is basic economy?
- isRefundable: Is the ticket refundable?
- isNonStop: Is the flight non-stop?
- baseFare: Price of the ticket (not including taxes).
- totalFare: Price of the ticket, including taxes and fees.
- seatsRemaining: Number of remaining seats.
- travelDistance: The total travel distance in miles.

In [2]:
case class FlightData(
    flightID:String,
    searchDate:String,
    searchMonth:String,
    searchDay:String,
    flightDate:String,
    flightMonth:String,
    flightDay:String,
    startingAirport:String,
    destinationAirport:String,
    duration:Int,
    isEconomy:Boolean,
    isRefundable:Boolean,
    isNonStop:Boolean,
    baseFare:Double,
    totalFare:Double,
    seatsRemaining:Int,
    travelDistance:Int
)

object FlightData {

    def parse(line:String) = {
        val input = line.split(",")
        val flightID = input(0)
        val searchDate = input(1)
        val searchMonth = searchDate.substring(5,7)
        val searchDay = searchDate.substring(8,10)
        val flightDate = input(2)
        val flightMonth = flightDate.substring(5,7)
        val flightDay = flightDate.substring(8,10)
        val startingAirport = input(3)
        val destinationAirport = input(4)
        val dur = input(6).replace("P","").replace("T","").split("D|H|M").map(x => x.toInt)
        val duration = dur.length match {
            case 3 => dur(0) * 1440 + dur(1) * 60 + dur(2)
            case 2 => dur(0) * 60 + dur(1)
            case 1 => dur(0)
        }
        val isEconomy = input(8).toBoolean
        val isRefundable = input(9).toBoolean
        val isNonStop = input(10).toBoolean
        val baseFare = input(11).toDouble
        val totalFare = input(12).toDouble
        val seatsRemaining = input(13).toInt
        val travelDistance = input(14) match {
            case "" => 0
            case _ => input(14).toInt
        }
        
        new FlightData(flightID,searchDate,searchMonth,searchDay,flightDate,flightMonth,flightDay,startingAirport,
                       destinationAirport,duration,isEconomy,isRefundable,isNonStop,baseFare,totalFare,
                       seatsRemaining,travelDistance)
    }
}

/** Create RRD */
val rdd = sc.textFile("s3a://"+bucketname+"/datasets/itineraries.csv")
/** Extract header from RDD and parse every row*/
val header = rdd.first(); 
val rddFlights = rdd.filter(row => row != header).map(FlightData.parse)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

defined class FlightData
defined object FlightData
Companions must be defined together; you may wish to use :paste mode for this.
rdd: org.apache.spark.rdd.RDD[String] = s3a://unibo-bd2223-vfolin/datasets/itineraries.csv MapPartitionsRDD[1] at textFile at <console>:29
header: String = legId,searchDate,flightDate,startingAirport,destinationAirport,fareBasisCode,travelDuration,elapsedDays,isBasicEconomy,isRefundable,isNonStop,baseFare,totalFare,seatsRemaining,totalTravelDistance,segmentsDepartureTimeEpochSeconds,segmentsDepartureTimeRaw,segmentsArrivalTimeEpochSeconds,segmentsArrivalTimeRaw,segmentsArrivalAirportCode,segmentsDepartureAirportCode,segmentsAirlineName,segmentsAirlineCode,segmentsEquipmentDescription,segmentsDurationInSeconds,segmentsDistance,segmentsCabinCode
rddFlights: org.apache.spark.rdd.RDD[FlightData] = MapPartitionsRDD[3] at map at <console>:29


#### Visualize data as a DataFrame

In [3]:
import spark.implicits._

val columns = Seq("flightID",
                  "searchDate",
                  "searchMonth",
                  "searchDay",
                  "flightDate",
                  "flightMonth",
                  "flightDay",
                  "startingAirport",
                  "destinationAirport",
                  "duration",
                  "isEconomy",
                  "isRefundable",
                  "isNonStop",
                  "baseFare",
                  "totalFare",
                  "seatsRemaining",
                  "travelDistance")

val flightDataframe = rddFlights.toDF(columns:_*)
flightDataframe.show(60,truncate=40,vertical=true)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import spark.implicits._
columns: Seq[String] = List(flightID, searchDate, searchMonth, searchDay, flightDate, flightMonth, flightDay, startingAirport, destinationAirport, duration, isEconomy, isRefundable, isNonStop, baseFare, totalFare, seatsRemaining, travelDistance)
flightDataframe: org.apache.spark.sql.DataFrame = [flightID: string, searchDate: string ... 15 more fields]
-RECORD 0----------------------------------------------
 flightID           | 9ca0e81111c683bec1012473feefd28f 
 searchDate         | 2022-04-16                       
 searchMonth        | 04                               
 searchDay          | 16                               
 flightDate         | 2022-04-17                       
 flightMonth        | 04                               
 flightDay          | 17                               
 startingAirport    | ATL                              
 destinationAirport | BOS                              
 duration           | 149                              
 isEc

#### Define a function to compute the number of days between the searchDate and the flightDate

In [4]:
import java.time.temporal.ChronoUnit.DAYS
import java.time.LocalDate

def daysBetween(x: FlightData): Long = {
        val searchDate = LocalDate.parse(x.searchDate);
        val flightDate = LocalDate.parse(x.flightDate);
        val daysBetween = DAYS.between(searchDate, flightDate);
        daysBetween
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import java.time.temporal.ChronoUnit.DAYS
import java.time.LocalDate
daysBetween: (x: FlightData)Long


## Data Exploration

#### Cache the dataset to run futher analysis and queries later

In [5]:
val rddFlightsCached = rddFlights.cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

rddFlightsCached: rddFlights.type = MapPartitionsRDD[3] at map at <console>:29


#### Perform the following counts to explore the dataset:

- Number of searches performed 
- Number of distinct flights
- Number of distinct startingAirport

In [7]:
"Number of searches performed: " + rddFlightsCached.count() //82138753
"Number of distinct flights: " + rddFlightsCached.map(x => x.flightID).distinct().count() // 5999739
"Number of distinct startingAirport: " + rddFlightsCached.map(x => (x.startingAirport)).distinct().count() //16

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res14: String = Number of searches performed: 82138753
res15: String = Number of distinct flights: 5999739
res16: String = Number of distinct startingAirport: 16


- Number of flights by starting airport
- Number of flights by month
- Number of routes, that are the possible combinations of starting and destination airport

In [19]:
print("Number of flights by starting airport: ")
val flightsByAirport = rddFlightsCached.
    map(x => (x.flightID, x.startingAirport)).
    distinct().
    map(x => (x._2, 1)).
    reduceByKey(_+_).
    collect()
// Array((LAX,580153), (CLT,394302), (JFK,288128), (BOS,439584), (OAK,309963), (LGA,426296), (ATL,362460), (MIA,343418), 
//(DTW,335334), (PHL,358076), (SFO,514561), (EWR,283379), (ORD,360045), (DEN,324702), (IAD,301958), (DFW,377531))

print("Number of flights by month: ") 
val flightsByMonth = rddFlightsCached.
    map(x => (x.flightID, x.flightMonth)).
    distinct().
    map(x => (x._2, 1)).
    reduceByKey(_+_).
    collect()
// Array((04,195651), (05,715546), (06,970381), (07,1004760), (08,1035367), (09,963734), (10,752287), (11,362013))

"Number of routes: " + rddFlightsCached.
    map(x => (x.startingAirport, x.destinationAirport)).
    distinct().
    count()
// 235

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Number of flights by month: flightsByMonth: Array[(String, Int)] = Array((04,195651), (05,715546), (06,970381), (07,1004760), (08,1035367), (09,963734), (10,752287), (11,362013))


- Number of flights that have an available economy ticket
- Number of flights that have an available no-economy ticket
- Number of direct flights

In [21]:
"Number of flights that have an available economy ticket: " + rddFlightsCached.
    filter(_.isEconomy).
    map(x => x.flightID).
    distinct().
    count()
// 1189124

"Number of flights that have an available non economy ticket: " + rddFlightsCached.
    filter(!_.isEconomy).
    map(x => x.flightID).
    distinct().
    count()
// 5551777

"Number of direct flights: " + rddFlightsCached.
    map(x => (x.flightID, x.isNonStop)).
    distinct().
    filter(x => x._2 == true).
    count() 
// 651615

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res92: String = Number of flights that have an available economy ticket: 1189124
res94: String = Number of flights that have an available non economy ticket: 5551777
res96: String = Number of direct flights: 651615


## Jobs

In [24]:
// define the path where we will save the results of queries
import org.apache.spark.sql.SaveMode
val path_output = "s3a://"+bucketname+"/spark/bdexam"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.sql.SaveMode
path_output: String = s3a://unibo-bd2223-vfolin/spark/bdexam


### 1 - Veronika Folin

The change that occurs in the ticket price in 14 days prior to flight departure is calculated, dividing the result between economy and no-economy tickets.

The correlation between that change in price and the number of seats remaining on that flight is then displayed.

#### We filter data to obtain searches made in the last 30 days before flight departure

The filtering operation allows us to reduce the number of records and therefore to optimize subsequent queries.

In [22]:
val rddFlightsFiltered = rddFlights.
    filter(x => daysBetween(x) <= 30).
    cache()

//"Number of filtered flights: " + rddFlightsFiltered.count()
// 47674280

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

rddFlightsFiltered: org.apache.spark.rdd.RDD[FlightData] = MapPartitionsRDD[132] at filter at <console>:33


#### We divide the result between economy and no-economy tickets

In [23]:
val rddEconomyFlights = rddFlightsFiltered.filter(_.isEconomy)
val rddNoEconomyFlights = rddFlightsFiltered.filter(!_.isEconomy)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

rddEconomyFlights: org.apache.spark.rdd.RDD[FlightData] = MapPartitionsRDD[133] at filter at <console>:30
rddNoEconomyFlights: org.apache.spark.rdd.RDD[FlightData] = MapPartitionsRDD[134] at filter at <console>:30


#### We calculate the average ticket price in a given number of days before departure

In [26]:
val rddAveragePriceEconomy = rddEconomyFlights.
    map(x => (daysBetween(x), x.baseFare)).
    aggregateByKey((0.0,0.0))((x,y)=>(x._1+y, x._2 + 1) , (x,y)=>(x._1 + y._1, x._2 + y._2)).
    map({case(k,v) => (k, v._1/v._2)})

val rddAveragePriceNoEconomy = rddNoEconomyFlights.
    map(x => (daysBetween(x), x.baseFare)).
    aggregateByKey((0.0,0.0))((x,y)=>(x._1+y, x._2 + 1) , (x,y)=>(x._1 + y._1, x._2 + y._2)).
    map({case(k,v) => (k, v._1/v._2)})

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

rddAveragePriceEconomy: org.apache.spark.rdd.RDD[(Long, Double)] = MapPartitionsRDD[137] at map at <console>:36
rddAveragePriceNoEconomy: org.apache.spark.rdd.RDD[(Long, Double)] = MapPartitionsRDD[140] at map at <console>:37


#### We calculate the average number of seats remaining on flights over a given number of days before departure

In [27]:
val rddAverageSeatsRemainingEconomy = rddEconomyFlights.
    map(x => (daysBetween(x), x.seatsRemaining)).
    aggregateByKey((0.0,0.0))((x,y)=>(x._1+y, x._2 + 1) , (x,y)=>(x._1 + y._1, x._2 + y._2)).
    map({case(k,v) => (k, v._1/v._2)})

val rddAverageSeatsRemainingNoEconomy = rddNoEconomyFlights.
    map(x => (daysBetween(x), x.seatsRemaining)).
    aggregateByKey((0.0,0.0))((x,y)=>(x._1+y, x._2 + 1) , (x,y)=>(x._1 + y._1, x._2 + y._2)).
    map({case(k,v) => (k, v._1/v._2)})

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

rddAverageSeatsRemainingEconomy: org.apache.spark.rdd.RDD[(Long, Double)] = MapPartitionsRDD[143] at map at <console>:36
rddAverageSeatsRemainingNonEconomy: org.apache.spark.rdd.RDD[(Long, Double)] = MapPartitionsRDD[146] at map at <console>:37


#### We join average ticket prices with average number of remaining seats, and then we save the result on S3 to reuse it

In [28]:
val rddEconomyResult = rddAverageSeatsRemainingEconomy.join(rddAveragePriceEconomy).
    map({case(k,v) => (k, v._1, v._2)}).
    coalesce(1).toDF().write.format("csv").mode(SaveMode.Overwrite).save(path_output)
val rddNoEconomyResult = rddAverageSeatsRemainingNonEconomy.join(rddAveragePriceNoEconomy).
    map({case(k,v) => (k, v._1, v._2)}).
    coalesce(1).toDF().write.format("csv").mode(SaveMode.Overwrite).save(path_output)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

rddEconomyResult: Unit = ()
rddNoEconomyResult: Unit = ()


#### We calculate the correlation matrix between the attributes AverageSeatsRemaining and AveragePrice

In [11]:
import org.apache.spark.mllib.stat.Statistics
import org.apache.spark.mllib.linalg._

val economyData = rddEconomyResult.map({case(days,(averageSeatsRemaining, averagePrice)) => 
    Vectors.dense(averageSeatsRemaining, averagePrice)}).cache()
val correlMatrixEconomy: Matrix = Statistics.corr(economyData)

val noEconomyData = rddNonEconomyResult.map({case(days,(averageSeatsRemaining, averagePrice)) => 
    Vectors.dense(averageSeatsRemaining, averagePrice)}).cache()
val correlMatrixNoEconomy: Matrix = Statistics.corr(noEconomyData)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.mllib.stat.Statistics
import org.apache.spark.mllib.linalg._
economyData: org.apache.spark.rdd.RDD[org.apache.spark.mllib.linalg.Vector] = MapPartitionsRDD[34] at map at <console>:36
correlMatrix: org.apache.spark.mllib.linalg.Matrix =
1.0                  -0.8847960167193927
-0.8847960167193927  1.0
nonEconomyData: org.apache.spark.rdd.RDD[org.apache.spark.mllib.linalg.Vector] = MapPartitionsRDD[43] at map at <console>:36
correlMatrix: org.apache.spark.mllib.linalg.Matrix =
1.0                  -0.5823660225408362
-0.5823660225408362  1.0


### 2 - Paolo Penazzi

We want to identify the cheapest flights departing in a given period, grouped according to the departure airport, calculating the following values:
- average price recorded in the 14 days before the departure of direct flights only
- previous result compared to the distance traveled by the flight, so we find the flights that take us as far as possible for less money

#### We filter data to obtain searches made in the last 14 days before flight departure and we discard the records for which the value of the travelDistance is equal to 0

The filtering operation allows us to reduce the number of records and therefore to optimize subsequent queries.

In [35]:
val rddFlightsFiltered = rddFlights.
    filter(x => daysBetween(x) <= 14).
    filter(_.travelDistance != 0).
    cache()

"Number of filtered flights: " + rddFlightsFiltered.count()
// 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

rddData1WeekBeforeDirectFlight: org.apache.spark.rdd.RDD[FlightData] = MapPartitionsRDD[148] at filter at <console>:45


In [36]:
def computeWeek(flightDate: String): Long = {
    val firstDate = LocalDate.parse("2022-04-16");
    val actualDate = LocalDate.parse(flightDate);
    DAYS.between(firstDate, actualDate)/7.toInt;
}
    

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

computeWeek: (flightDate: String)Long


In [37]:
val rddFlightsWithPeriod = rddFlightsFiltered.
    map(x => (x.flightID, x.flightDate)).
    distinct().
    map(x => (x._1, computeWeek(x._2))).
    collect()
    

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

rddFlightsWithPeriod: Array[(String, Long)] = Array((dd4aa677b91bf7189dbfe2bebd4390fa,8), (2ba8ee35da99220fa49c4788ad14e510,15), (ebc67e6882e6edeec41c2bb5ef36af19,15), (b99efbe8fc2fa32d75b161d2ddda65ad,9), (217fa11b56b8e19644a7349b969f5c9d,8), (3701f54e60459dfd11388c996a8e2c3b,17), (082789de8f1692e3a77884655f3c25b5,12), (a32167a41cbb6fd8b37d493f9816523f,11), (a957e5cb8b5208d297da93e2602f4b79,15), (daa584065c234debfdd34ba0ab676b91,10), (ddc74c2434de9a0a9504590399e348cd,17), (d9839b5698275121f5f0c364f54747f8,5), (ad7383eff7fcb51ee0b736e37c975fa5,9), (98075cfdb54d46636f7bcb309c51d38a,12), (3da0dc5274a172779ab142e6bbd97dfa,6), (68e0e1c4db5bf46c75bfd3099da51b5f,25), (8219ab46c02b48b5ca8433ae8cab4389,8), (ca5fc364a652dc89be5ef58d1bda1ff8,28), (f1280ef17b6d4c0d6bd8667c5a10bb54,1), (698e3f140df...

In [36]:
val rddAveragePricePerFlight = rddFlightsFiltered.
    map(x => (x.flightID, x.baseFare)).
    aggregateByKey((0.0,0.0))((x,y)=>(x._1+y, x._2 + 1) , (x,y)=>(x._1 + y._1, x._2 + y._2)).
    map({case(k,v) => (k, v._1/v._2)})

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

rddAveragePricePerFlight: org.apache.spark.rdd.RDD[(String, Double)] = MapPartitionsRDD[151] at map at <console>:43


In [37]:
val rddAveragePricePerFlightWithDistance = rddData1WeekBeforeDirectFlight.
    map(x => (x.flightID, x.travelDistance)).
    distinct().
    join(rddAveragePricePerFlight)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

rddAveragePricePerFlightWithDistance: org.apache.spark.rdd.RDD[(String, (Int, Double))] = MapPartitionsRDD[158] at join at <console>:45


In [38]:
val rddAveragePricePerDistance = rddAveragePricePerFlightWithDistance.
    map({case(x,y) => (x, y._2/y._1)})

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

rddAveragePricePerDistance: org.apache.spark.rdd.RDD[(String, Double)] = MapPartitionsRDD[159] at map at <console>:41


In [39]:
val rddAveragePricePerAirport = rddData1WeekBeforeDirectFlight.
    map(x => (x.flightID, x.startingAirport)).
    distinct().
    join(rddAveragePricePerDistance)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

rddAveragePricePerAirport: org.apache.spark.rdd.RDD[(String, (String, Double))] = MapPartitionsRDD[166] at join at <console>:45


In [45]:
val rddFinal = rddAveragePricePerAirport.
    map({case(x,y) => (y._1, y._2)}).
    aggregateByKey((0.0,0.0))((x,y)=>(x._1+y, x._2 + 1) , (a,b)=>(a._1 + b._1, a._2 + b._2)).
    map({case(k,v) => (v._1/v._2, k)}).
    sortByKey()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

rddFinal: org.apache.spark.rdd.RDD[(Double, String)] = ShuffledRDD[181] at sortByKey at <console>:44


In [47]:
val df = rddFinal.toDF("Airport", "AvgPricePerDistance")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

df: org.apache.spark.sql.DataFrame = [Airport: double, AvgPricePerDistance: string]


In [69]:
import $ivy.`org.vegas-viz::vegas_2.11:0.3.11`


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
<console>:38: error: not found: value $ivy
       import $ivy.`org.vegas-viz::vegas_2.11:0.3.11`
              ^



In [65]:
import `org.vegas-viz:vegas_2.11:0.3.11`
import `org.vegas-viz:vegas-spark_2.11:0.3.11`

import vegas._
import vegas.render.WindowRenderer._
import vegas.sparkExt._

Vegas(“Average price/distance per airport”)
 .withDataFrame(df)
 .mark(Bar) // Change to .mark(Area)
 .encodeX(“Airport”, Nom)
 .encodeY(“AvgPricePerDistance”, Quant)
 .show

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
<console>:2: error: '.' expected but ';' found.
import `org.vegas-viz:vegas-spark_2.11:0.3.11`
^

